# RAG Model for Junior AI Engineer

This notebook is an end-to-end walkthrough of a RAG pipeline, from the indexing, to the query until demo deployment on Gradio. It is a simple RAG pipeline set to answer questions on the user based on their data, in my case I'm making use of my resume of course, among other things!


## Preparing the notebook

This project was mainly done on Google Colaboratory and as such have some code specific to it. As seen below the notebook must have the user's drive mounted or connected to it to be able to access their storage.

Google Drive was utilized to store necessary documents to be processed (i.e. Resume, Cover Letter)

In [ ]:
#Mounting the drive to access data storage
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Next, to help in debugging we import logging. This is recommend by Haystack (the open source framework we will be using) for the implementation of their modules.

In [ ]:
#For better debugging using Haystack
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s",
                    level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.ERROR)

## Packages and Dependencies

Let's install the necessary dependencies. We'll be making use of the following packages.

One of the first decisions to make when creating a RAG pipeline is what framework to use, as mentione dearlier we will be using haystack.

In [ ]:
%%bash

pip install haystack-ai
pip install "sentence-transformers>=3.0.0"
pip install google-ai-haystack
pip install markdown-it-py mdit_plain pypdf
pip install gdown
pip install trafilatura
pip install chroma-haystack
pip install gradio

Next we start importing the necessary modules. The first import from google is important as it is how we will be accessing Colab's secrets.

As we will see later there will be instances where API Keys will be necessary, but we don't want them to be visible in the code. There are other solutions to this, but given that Colab has the secrets function we will utilize that.

In [ ]:
# Import this if you're using Colab so you can retrieve stored Secrets
from google.colab import userdata

import os

from haystack import Pipeline
from haystack.components.converters import HTMLToDocument
from haystack.components.writers import DocumentWriter
from haystack.components.converters import MarkdownToDocument, PyPDFToDocument, TextFileToDocument
from haystack.components.preprocessors import DocumentSplitter, DocumentCleaner
from haystack.components.routers import FileTypeRouter
from haystack.components.joiners import DocumentJoiner
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.fetchers import LinkContentFetcher
from haystack_integrations.document_stores.chroma import ChromaDocumentStore

/usr/local/lib/python3.10/dist-packages/haystack/core/errors.py:34: DeprecationWarning: PipelineMaxLoops is deprecated and will be remove in version '2.7.0'; use PipelineMaxComponentRuns instead.
  warnings.warn(


# Building the Indexing Pipeline

We start with an Indexing Pipeline, where we will take our data, convert them into a format our AI will understand, and place them into a storage.

Haystack offers several modules, for our purposes will be making use of ChromaDB as can be seen in this import:

`from haystack_integrations.document_stores.chroma import ChromaDocumentStore`

## Setting up your data storage

Since we're using Colab we've got our PDFs on Google Drive, and so we set the folder we've got them on as the path.

With this our ChromaDB Document Store is set to place our pre-processed documents in `chromaDB_path`


In [ ]:
# %cd drive/MyDrive/'Colab Notebooks'
chromaDB_path = "/content/drive/MyDrive/Colab Notebooks/OBZ Exam/Data Storage/chromaDB"
document_store = ChromaDocumentStore(persist_path=chromaDB_path)

## File Handling

While the files in the storage are primarily pdfs, we want to ensure our model is able to handle different types of input. We'll be using `FileTypeRouter` from Haystack which routes files paths or byte streams based on their type to the appropriate output for processing.

In [ ]:
file_type_router = FileTypeRouter(mime_types=["text/plain",
                                              "application/pdf",
                                              "text/markdown"])

The files will be routed to one of these converters:

Converters extract the data from the file and convert them to a document format that haystack makes use of. Then the document_joiner takes their outputs and unifies them so that only a single output goes towards the cleaner.

In [ ]:
text_file_converter = TextFileToDocument()
markdown_converter = MarkdownToDocument()
pdf_converter = PyPDFToDocument()
html_to_document= HTMLToDocument()
document_joiner = DocumentJoiner()

For cleaning these documents Haystack provides a DocumentCleaner function. It also provides for a document splitter which divides the documents into lists of shorter text documents so that our LLMs can process them faster.

Our documents are not that lengthy or we split them into small chunks and with some overlap just so that we maintains some semantic context between words.

In [ ]:
document_cleaner = DocumentCleaner()
document_splitter = DocumentSplitter(
    split_by="word",
    split_length=150,
    split_overlap=50
    )

Next we need an embedder because our LLMs don't actually read the words but their vector representation, I've kept it simple and used the default model Haystack tends to use in their documentation.

Finally, we need a document_writer or a function that will place these documents into our document store.

In [ ]:
document_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
document_writer = DocumentWriter(document_store)

## Putting it all together

We put the pipeline together, by instantiating it as components:

In [ ]:
preprocessing_pipeline = Pipeline()
preprocessing_pipeline.add_component(instance=file_type_router,
                                     name="file_type_router")
preprocessing_pipeline.add_component(instance=text_file_converter,
                                     name="text_file_converter")
preprocessing_pipeline.add_component(instance=markdown_converter,
                                     name="markdown_converter")
preprocessing_pipeline.add_component(instance=pdf_converter,
                                     name="pypdf_converter")
preprocessing_pipeline.add_component(instance=html_to_document,
                                     name="html_to_document")
preprocessing_pipeline.add_component(instance=document_joiner,
                                     name="document_joiner")
preprocessing_pipeline.add_component(instance=document_cleaner,
                                     name="document_cleaner")
preprocessing_pipeline.add_component(instance=document_splitter,
                                     name="document_splitter")
preprocessing_pipeline.add_component(instance=document_embedder,
                                     name="document_embedder")
preprocessing_pipeline.add_component(instance=document_writer,
                                     name="document_writer")

We then connect them, which is done by taking the output of one component and placing it into the input of another.

It should be important to note here that Haystack only allows one component to be connected to another component, meaning I can't connect `document_joiner` to `document_cleaner` and then `document_splitter` at the same time.

There are exceptions for `document_joiner` where it takes several outputs from our different converters.

In [ ]:
#Connect the components, output to input
preprocessing_pipeline.connect("file_type_router.text/plain", "text_file_converter.sources")
preprocessing_pipeline.connect("file_type_router.application/pdf", "pypdf_converter.sources")
preprocessing_pipeline.connect("file_type_router.text/markdown", "markdown_converter.sources")
preprocessing_pipeline.connect("file_type_router.text/plain", "html_to_document.sources")
# preprocessing_pipeline.connect("link_content_fetcher.streams", "html_to_document_link")
preprocessing_pipeline.connect("text_file_converter", "document_joiner")
preprocessing_pipeline.connect("html_to_document", "document_joiner")
preprocessing_pipeline.connect("pypdf_converter", "document_joiner")
preprocessing_pipeline.connect("markdown_converter", "document_joiner")
preprocessing_pipeline.connect("document_joiner", "document_cleaner")
preprocessing_pipeline.connect("document_cleaner", "document_splitter")
preprocessing_pipeline.connect("document_splitter", "document_embedder")
preprocessing_pipeline.connect("document_embedder", "document_writer")

🚅 Components
  - file_type_router: FileTypeRouter
  - text_file_converter: TextFileToDocument
  - markdown_converter: MarkdownToDocument
  - pypdf_converter: PyPDFToDocument
  - html_to_document: HTMLToDocument
  - document_joiner: DocumentJoiner
  - document_cleaner: DocumentCleaner
  - document_splitter: DocumentSplitter
  - document_embedder: SentenceTransformersDocumentEmbedder
  - document_writer: DocumentWriter
🛤️ Connections
  - file_type_router.text/plain -> text_file_converter.sources (List[Path])
  - file_type_router.application/pdf -> pypdf_converter.sources (List[Path])
  - file_type_router.text/markdown -> markdown_converter.sources (List[Path])
  - file_type_router.text/plain -> html_to_document.sources (List[Path])
  - text_file_converter.documents -> document_joiner.documents (List[Document])
  - markdown_converter.documents -> document_joiner.documents (List[Document])
  - pypdf_converter.documents -> document_joiner.documents (List[Document])
  - html_to_document.docu

We run the pipeline and our very first component requires an input, in this case it is the folder on my google drive where the pdfs are stored.

We use the `glob()` method to get any  and all files and directories inside the folder.

We run the pipeline and find ourselves with 11 documents written and placed in our document store.

In [ ]:
from pathlib import Path
data_path = "/content/drive/MyDrive/Colab Notebooks/OBZ Exam/Data Storage"
preprocessing_pipeline.run(
    {"file_type_router": {"sources": list(Path(data_path).glob("**/*"))}}
    )

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'file_type_router': {'unclassified': [PosixPath('/content/drive/MyDrive/Colab Notebooks/OBZ Exam/Data Storage/a8i9w57wqoyi8fjxs9ulzk14e'),
   PosixPath('/content/drive/MyDrive/Colab Notebooks/OBZ Exam/Data Storage/2oz8eti4z4twooufhuoruh8rw'),
   PosixPath('/content/drive/MyDrive/Colab Notebooks/OBZ Exam/Data Storage/d9j378s9zze513ts3noxdx3lf'),
   PosixPath('/content/drive/MyDrive/Colab Notebooks/OBZ Exam/Data Storage/2qo06b0yji7gk9r0emy6ews0e'),
   PosixPath('/content/drive/MyDrive/Colab Notebooks/OBZ Exam/Data Storage/7vion8srg89ffkc518s5kgvg5'),
   PosixPath('/content/drive/MyDrive/Colab Notebooks/OBZ Exam/Data Storage/bpjozp1uyx91l2qz29pdgjcpw.css'),
   PosixPath('/content/drive/MyDrive/Colab Notebooks/OBZ Exam/Data Storage/4pi7vivqtppd21qxirh03sub0'),
   PosixPath('/content/drive/MyDrive/Colab Notebooks/OBZ Exam/Data Storage/3dwppu0c34e20ignenu8ihgt7.css'),
   PosixPath('/content/drive/MyDrive/Colab Notebooks/OBZ Exam/Data Storage/y41qzg45cwdddxb07wfr97dx'),
   PosixPath('/conten

# Building the Query Pipeline

Now that our Indexing Pipeline is setup, we move onto creating our Query Pipeline, where we setup the LLM that will interacting with our data storage, and answering questions based on the context we provide.

## Embedder

We set up an embedder to convert our documents into vector representations, which our LLM (Large Language Model) can process and understand. The embedder we're using, the SentenceTransformersTextEmbedder, requires a Hugging Face API key since the model is hosted on Hugging Face's platform. I securely stored my API key in the environment and accessed it as an environment variable, ensuring smooth access to the embedding model.

you can run the `warm_up()` to see if the embedder is working properly.

In [ ]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.embedders import SentenceTransformersTextEmbedder

os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
embedder = SentenceTransformersTextEmbedder(
    model="sentence-transformers/all-MiniLM-L6-v2"
    )
# embedder.warm_up()

## Retriever

Our choice of retriever is limited as we have to match it to our choice of data storage, in this case we make use of the `ChromaEmbeddingRetriever`.

In [ ]:
from haystack_integrations.components.retrievers.chroma import ChromaEmbeddingRetriever
retriever = ChromaEmbeddingRetriever(document_store=document_store)

## Template

We need to build prompts for interacting with LLMs  and so we import the `PromptBuilder` from haystack.

It makes use of the Jinja Template as its structure, it is a set of placeholders that are filled when the the template is used. The full documentation for Jinja can be found [here](https://jinja.palletsprojects.com/en/3.0.x/templates/)

In [ ]:
from haystack.components.builders import PromptBuilder

template = """
You are designed to answer questions about a potential candidate for the position of
Junior AI Engineer at OneByZero. Use the information provided to you to answer these questions
to the best of your ability to speak about the candidate.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{ question }}
Answer:
"""

prompt_builder = PromptBuilder(template=template)

# Generator

Haystack makes use of several llms, seeing as we're working primarily on Colab, I'll be making use of Gemini (that and it's free, sorry OpenAI). Specifically I made use of `gemini-1.5-flash` model.

We import the `GoogleAIGeminiGenerator from Haystack, setup the necessary API Key and place the model in its parameters.

In [ ]:
#Remember to get the Gemini dependency
from haystack_integrations.components.generators.google_ai import GoogleAIGeminiGenerator
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

generator = GoogleAIGeminiGenerator(model="gemini-1.5-flash")

## Putting the Query Pipeline together

With all the parts ready we instantiate and connec them like our Indexing Pipeline.

In [ ]:
#Instantiate the RAG pipeline
query_pipeline = Pipeline()
query_pipeline.add_component("embedder", embedder)
query_pipeline.add_component("retriever", retriever)
query_pipeline.add_component("prompt_builder", prompt_builder)
query_pipeline.add_component("llm", generator)

#connect them
query_pipeline.connect("embedder.embedding", "retriever.query_embedding")
query_pipeline.connect("retriever", "prompt_builder.documents")
query_pipeline.connect("prompt_builder.prompt", "llm")

🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - retriever: ChromaEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: GoogleAIGeminiGenerator
🛤️ Connections
  - embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.parts (str)

The whole pipeline is complete at this point, and we can test it out. Let's start by asking something simple like who is the job candidate.

In [ ]:
question = (
    "Who is the candidate for the position?"
    )
response = query_pipeline.run(
    {
        "embedder": {"text": question},
        "prompt_builder": {"question": question}
    }
)
response["llm"]["replies"][0]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'The candidate for the Junior AI Engineer position at OneByZero is **Joshua Victor C. San Juan**. \n'

That's a bit hard to read, but it got the job done. We're ready to place it on our demo application.

For the purposes of this demonstration we'll be making use of Gradio as I found it to be most compatible with a Haystack pipeline.

However, before we place our pipeline onto Gradio we just need to add one last feature.

## Chat history

If you go back to our initial `query_pipeline` and ask it to tell you what the previous question was, our model will not be able to answer because it is not part of its context window.

To account for this we add a `chat_history` to store the previous messages and adjust our pipeline a little to account for this, we start referring to it as a prompt since, we account not just for the question but also the chat history.

In [ ]:
# Initialize an empty list to store chat history
chat_history = []

def generate_answer(message, history):
    # Add the current message to the chat history
    chat_history.extend([("user", message)])

    # Include history in the prompt (adjust as needed)
    prompt = f"""
    ## Chat History:
    {format_history(history)}

    ## User's Question:
    {message}
    """

    result = query_pipeline.run(
        {
            "embedder": {"text": str(prompt)},  # Embed the prompt with history
            "prompt_builder": {"question": prompt}
        }
    )

    answer = result["llm"]["replies"][0]

    # Add the answer to the chat history
    chat_history.extend([("assistant", answer)])

    # Return only the answer, Gradio handles history
    return answer  # or answer, history

def format_history(history):
    """Formats the chat history for the prompt."""
    formatted_history = ""
    for role, content in history:
        formatted_history += f"{role.capitalize()}: {content}\n"
    return formatted_history

# Deployment Demo with Gradio

The model is ready to be deployed and thankfully Gradio provides a `ChatInterface()` to go with our very own RAG AI application.

Let's import it first, and then set it up.


In [ ]:
#Make sure to import it first
import gradio as gr

#Setting up the Chatbot Interface
chatbot_with_gemini = gr.ChatInterface(
    generate_answer,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder='Ask me a question about this candidate', container=False, scale=7),
    title="RAG AI Chatbot powered by Gemini",
    description="Ask me about Osh and his qualifications",
    theme="soft",
    examples=[
        "What is his work experience?",
        "What is his education?",
        "What are his skills?",
    ],
    cache_examples=False,
    submit_btn="Ask",
    multimodal=True
)

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:222: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


A lot of the parameters are mostly for aesthetic reasons, what's important for this is the first one, `generate_answer` which is our function that calls on the LLM to answer the queries.

Now let's launch it and try it out.

We don't have to set `debut=True` but incase anything goes wrong, at least we'll have an idea of why it happened.

In [ ]:
chatbot_with_gemini.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6a4d984009c9cc1441.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


And there we have it a full RAG pipeline.

# Methods for Evaluation

To evaluate this pipeline, there are several metrics, I'll be taking mine from those suggested by Haystack [here](https://haystack.deepset.ai/tutorials/35_evaluating_rag_pipelines) ,as they do provide functions to create an evaluation pipeline (which I may add in the near future).



*   **Document Mean Reciprocal** : This evaluates the documents the model pulls from the storage and checks how they were ranked.
*   **Semantic Answer Similarity**: This checks if the answer provided shares similar semantics to the document it pulled.
*   **Faithfulness**: Makes use of an LLM to check if the answer can be inferred from the context (does not need ground truth labels






